In [1]:
import numpy as np # linear algebra
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPool1D
from keras.layers import Bidirectional

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv('./preprocessing/train.csv')
test_df = pd.read_csv('./preprocessing/test.csv')

In [3]:
train_text = train_df.text.astype(str)
test_text = test_df.text

In [4]:
max_features = 5000
tokens = Tokenizer(max_features)
tokens.fit_on_texts(train_text)
index_train = tokens.texts_to_sequences(train_text)
index_test = tokens.texts_to_sequences(test_text)

In [5]:
max_len = 200
tr_x = pad_sequences(index_train,maxlen = 200)
te_x = pad_sequences(index_test,maxlen = 200)

In [6]:
model = Sequential()
model.add(Embedding(input_dim = 5000,output_dim = 256,input_length = 200))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 256)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 200, 512)         1050624   
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 512)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 150)               76950     
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 3

In [7]:
import tensorflow.keras.optimizers as optimizers

model.compile(loss= 'binary_crossentropy',  optimizer=optimizers.Adam(learning_rate=.0001), metrics=[ 'accuracy' ])

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks = [
    EarlyStopping(patience=3, verbose=1),
    ReduceLROnPlateau(factor=0.25, patience=2, min_lr=0.00001, verbose=1),
    # ModelCheckpoint('model_lstm.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [9]:
from keras.utils import to_categorical

X = tr_x
y = train_df.target
y = to_categorical(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
model.fit(X_train, y_train, epochs=10, callbacks=callbacks, validation_data=(X_val,y_val))

Epoch 1/10
186/186 [==============================] - 315s 2s/step - loss: 0.6823 - accuracy: 0.5692 - val_loss: 0.6698 - val_accuracy: 0.5730 - lr: 1.0000e-04
Epoch 2/10
186/186 [==============================] - 327s 2s/step - loss: 0.6141 - accuracy: 0.6976 - val_loss: 0.5034 - val_accuracy: 0.7781 - lr: 1.0000e-04
Epoch 3/10
186/186 [==============================] - 326s 2s/step - loss: 0.4065 - accuracy: 0.8336 - val_loss: 0.4923 - val_accuracy: 0.7956 - lr: 1.0000e-04
Epoch 4/10
186/186 [==============================] - 319s 2s/step - loss: 0.3383 - accuracy: 0.8693 - val_loss: 0.4855 - val_accuracy: 0.7929 - lr: 1.0000e-04
Epoch 5/10
186/186 [==============================] - 318s 2s/step - loss: 0.2820 - accuracy: 0.8987 - val_loss: 0.5016 - val_accuracy: 0.7909 - lr: 1.0000e-04
Epoch 6/10
186/186 [==============================] - ETA: 0s - loss: 0.2460 - accuracy: 0.9146
Epoch 6: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
186/186 [===================

In [11]:
predictions = model.predict(te_x)
y_pred = predictions.argmax(axis = 1)

102/102 [==============================] - 36s 343ms/step


In [12]:
from sklearn.metrics import accuracy_score

ans = pd.read_csv('./dataset/ans.csv')['target'].values
accuracy_score(y_pred= y_pred, y_true= ans)

0.7894575543977934